<a href="https://colab.research.google.com/github/RafaelGallo/LLM/blob/main/LLM_Cluster_Kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Modelo 1 - KMEANS**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# Carregar dados de exemplo
data = [
    "O céu está azul.",
    "O sol está brilhando.",
    "As flores estão florescendo.",
    "Estou aprendendo sobre clusterização de dados.",
    "A programação é divertida.",
    "Python é uma linguagem de programação popular.",
]

In [ ]:
# Vetorização das palavras usando TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data)

In [ ]:
# Aplicando o algoritmo K-means
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(X)

In [ ]:
# Examinando os clusters
print("Palavras agrupadas por cluster:")
for i in range(num_clusters):
    cluster_words = []
    for j in range(len(data)):
        if kmeans.labels_[j] == i:
            cluster_words.append(data[j])
    print(f"Cluster {i+1}: {cluster_words}")

In [ ]:
from sklearn.decomposition import TruncatedSVD

# Reduzindo a dimensionalidade dos dados para visualização
lsa = TruncatedSVD(n_components=2)

# Treinamento modelo
X_reduced = lsa.fit_transform(X)

# Visualizando modelo
lsa

In [ ]:
# Plotando os clusters
plt.figure(figsize=(8, 6))
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
for i in range(num_clusters):
    cluster_points = X_reduced[kmeans.labels_ == i]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], c=colors[i], label=f'Cluster {i+1}')

plt.title('Clusters de Palavras - Kmeans')
plt.xlabel('Componente 1')
plt.ylabel('Componente 2')
plt.grid(False)
plt.legend()
plt.show()

# **Modelo 2 - LLM EleutherAI/gpt-neo-1.3B**

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

import logging
from transformers import pipeline

In [ ]:
# Configuração do logger para exibir apenas as mensagens de nível 'error'
logging.getLogger().setLevel(logging.ERROR)

In [ ]:
# Carregar o modelo e o tokenizador
# Substitua pelo modelo de sua escolha

# Modelo LLM
model_name = "EleutherAI/gpt-neo-1.3B"

In [ ]:
# Tokenização
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Adicionar token de padding
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Modelo
model = AutoModel.from_pretrained(model_name)

In [ ]:
# Carregar dados de exemplo
data = [
    "O céu está azul.",
    "O sol está brilhando.",
    "As flores estão florescendo.",
    "Estou aprendendo sobre clusterização de dados.",
    "A programação é divertida.",
    "Python é uma linguagem de programação popular.",
]


In [ ]:
# Obter embeddings para os dados
embeddings = []
for text in data:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())

# Convertendo embeddings para um formato aceitável para o método fit
embeddings = np.array(embeddings)

In [ ]:
# Encontrando o valor K

# Importando biblioteca
from yellowbrick.cluster import KElbowVisualizer

# Use o método do cotovelo para encontrar o valor de K ideal
visualizer = KElbowVisualizer(KMeans(), k=(2,6))
visualizer.fit(embeddings)
visualizer.show()

In [ ]:
# Análise de Sentimento
classifier = pipeline('sentiment-analysis')
sentimentos = classifier("embeddings")

# Visualizando os sentimentos LLM
sentimentos

In [ ]:
# Realizar a clusterização com k-means

# Número de clusters
k = 3

# Modelo Kmeans
kmeans = KMeans(n_clusters=k)

# Treinamento modelo
kmeans.fit(embeddings)

In [ ]:
# Labels kmeans
labels = kmeans.labels_

In [ ]:
# Agrupar os comentários nos clusters
comentarios_clustered = pd.DataFrame(kmeans.predict(embeddings), columns=['Cluster'])
comentarios_clustered.head()

In [ ]:
# Exibir os resultados
print("Comentários agrupados por cluster:")
print(comentarios_clustered)

In [ ]:
# Avaliar a qualidade dos clusters
silhouette_avg = silhouette_score(embeddings, kmeans.labels_)
print("\n Silhouette Score: %.2f" % silhouette_avg)

In [ ]:
# Reduzir a dimensionalidade para visualização
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings)

# Visualizando modelo PCA
pca

In [ ]:
# Plotar os clusters
colors = ['r', 'g', 'b', 'y', 'c', 'm']  # Cores para os clusters
for i in range(len(data)):
    plt.scatter(embeddings_2d[i, 0],
                embeddings_2d[i, 1],
                color=colors[labels[i]],
                label=f'Cluster {labels[i]}')

plt.title('Clusters - LLM EleutherAI/gpt-neo-1.3B')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend()
plt.show()

## **3) LLM modelo Google - BERT Análise de sentimento**

In [ ]:
from transformers import BertTokenizer, BertModel
from torch.nn import functional as F
import torch

In [ ]:
# Carregar o tokenizador do BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Definir uma frase de exemplo
text = "I love this book!"

In [ ]:
# Tokenizar a frase e converter para uma sequência de IDs
inputs = tokenizer.encode(text, return_tensors="pt")

In [ ]:
# Inicializar o modelo BERT
model = BertModel.from_pretrained('bert-base-uncased')
model

In [ ]:
# Fazer a previsão de sentimento
with torch.no_grad():
    outputs = model(inputs)

In [ ]:
# Análise de Sentimento
classifier = pipeline('sentiment-analysis')
sentimentos = classifier("inputs")

# Visualizando os sentimentos LLM
print()
print(sentimentos)

In [ ]:
# Obter a saída do último token de classificação
output = outputs.last_hidden_state[:, 0, :].squeeze()

In [ ]:
# Pegar a probabilidade do token de classificação mais provável
probability = F.softmax(output, dim=0)

In [ ]:
# Pegar o índice do token de classificação mais provável
predicted_index = torch.argmax(probability, dim=0).tolist()
predicted_index

# **4) Modelo LLM - BART Clusterização**

In [ ]:
import torch
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from transformers import BertTokenizer, BertModel

In [ ]:
# Função para obter os embeddings BERT de um texto
def get_bert_embeddings(texts):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    embeddings = []
    for text in texts:
        input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)
        with torch.no_grad():
            outputs = model(input_ids)
            last_hidden_states = outputs[0]
            pooled_output = last_hidden_states.mean(axis=1).squeeze()
            embeddings.append(pooled_output.numpy())

    return np.array(embeddings)

In [ ]:
# Dados de exemplo (textos)
texts = [
    "O Brasil é um país localizado na América do Sul.",
    "Paris é a capital da França.",
    "O futebol é um esporte popular em todo o mundo.",
    "Python é uma linguagem de programação poderosa.",
    "A Terra gira em torno do Sol."
]

In [ ]:
# Obter embeddings BERT para os textos
embeddings2 = get_bert_embeddings(texts)

In [ ]:
# Lista para armazenar os valores de inércia
inertia = []

# Testar diferentes valores de k
for k in range(1, 6):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(embeddings2)
    inertia.append(kmeans.inertia_)

# Plotar o gráfico Elbow
plt.plot(range(1, 6), inertia, marker='o')
plt.xlabel('Número de clusters')
plt.ylabel('Inércia')
plt.title('Método Elbow para encontrar o valor de k')
plt.show()

In [ ]:
# Modelo kmeans

# Determinar o valor de k usando o método Elbow
optimal_k = 2 # Este valor é o que você encontrou como ideal no gráfico Elbow

# Criar e ajustar o modelo K-means com o valor de k ideal
kmeans = KMeans(n_clusters=optimal_k)
kmeans.fit(embeddings2)

In [ ]:
# Obter os rótulos dos clusters para cada amostra
cluster_labels = kmeans.predict(embeddings2)
cluster_labels

In [ ]:
# Obter os índices dos clusters para cada texto e os centroides dos clusters
cluster_indices, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, embeddings2)

In [ ]:
# Obter os índices dos clusters para cada texto e os centroides dos clusters
cluster_indices = kmeans.predict(embeddings2)

# Imprimir os resultados
for i, text in enumerate(texts):
    print(f"Texto: {text} - Cluster: {cluster_indices[i]}")


In [ ]:
# Modelo PCA
pca = PCA(n_components=2)

# Treinamento modelo
embeddings_2d = pca.fit_transform(embeddings2)

# Visualizando modelo
pca

In [ ]:
# Visualizando o gráfico
plt.figure(figsize=(8, 6))
for i in range(optimal_k):
    plt.scatter(embeddings_2d[cluster_labels == i, 0],
                embeddings_2d[cluster_labels == i, 1],
                label=f'Cluster {i}')
plt.title('Clusters LLM - BART')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend()
plt.show()